In [1]:
from assets.credentials import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

from binance.client import Client

In [3]:
client = Client(api_key=BINANCE_KEY, api_secret=BIANCE_SECRET)

In [104]:
# prices = client.get_historical_klines(symbol='BTCUSDT', interval = Client.KLINE_INTERVAL_1MINUTE, start_str=1451628000000, limit = 1000)
futures_prices = client.futures_klines(symbol='BTCUSDT', interval = Client.KLINE_INTERVAL_1MINUTE, start_str=1451628000000)

In [106]:
# prices_array[0]

In [107]:
# df = pd.DataFrame()
# df['Time'] = prices_array[0].astype('int')
# df['Open'] = prices_array[1].astype('float')
# df['High'] = prices_array[2].astype('float')
# df['Low'] = prices_array[3].astype('float')
# df['Close'] = prices_array[4].astype('float')
# df['Volume'] = prices_array[5].astype('float')
# df['Trades'] = prices_array[8].astype('int')

In [4]:
df = pd.read_csv('assets/BTCUSDT_historical.csv')
df['Time'] = pd.to_datetime(df['Time'], unit = 'ms')

In [5]:
%matplotlib widget
plt.close('all')
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
plt.plot(df['Time'], df['Open'], label = 'Open')
plt.plot(df['Time'], df['High'], label = 'High')
plt.plot(df['Time'], df['Low'], label = 'Low')
plt.plot(df['Time'], df['Close'], label = 'Close')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.close('all')
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
plt.plot(df['Time'], df['Trades'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
df['minute'] = df['Time'].dt.minute
df['hour'] = df['Time'].dt.hour
df['day'] = df['Time'].dt.day
df['month'] = df['Time'].dt.month
df['year'] = df['Time'].dt.year
df = df[['Time', 'minute', 'hour', 'day', 'month', 'year', 'Open', 'High', 'Low', 'Close', 'Volume', 'Trades']]
df = df.sort_values("Time").drop(['Time'], axis = 1)
df = df.drop(['High', 'Low', 'Close', 'Volume', 'Trades'], axis = 1)
df

,minute,hour,day,month,year,Open
0,0,4,17,8,2017,4261.48
1,1,4,17,8,2017,4261.48
2,2,4,17,8,2017,4280.56
3,3,4,17,8,2017,4261.48
4,4,4,17,8,2017,4261.48
...,...,...,...,...,...,...
1836733,20,22,17,2,2021,52173.13
1836734,21,22,17,2,2021,52170.18
1836735,22,22,17,2,2021,52156.73
1836736,23,22,17,2,2021,52172.19


In [8]:
label_scalers = {}
train_x, train_y, test_x, test_y = None, None, None, None

In [13]:
df.shape

(1836738, 6)

In [14]:
data = df.values

In [15]:
lookback = 90
inputs = np.zeros((len(data)-lookback,lookback,df.shape[1]))
labels = np.zeros(len(data)-lookback)

In [16]:
for i in range(lookback, len(data)):
    inputs[i-lookback] = data[i-lookback:i]
    labels[i-lookback] = data[i,-1]
inputs = inputs.reshape(-1,lookback,df.shape[1])
labels = labels.reshape(-1,1)

In [17]:
test_portion = int(0.2*len(inputs))
train_x = inputs[:-test_portion]
train_y = labels[:-test_portion]
test_x = inputs[-test_portion:]
test_x = labels[-test_portion:]

In [19]:
print(train_y[0])
df['Open'].iloc[90]

[4320.]


4320.0

In [35]:
batch_size = 128
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle = True, batch_size = batch_size, drop_last = True)

for x, label in train_loader:
    print(x[0][-1])
    print(x.float()[0][-1])
#     print(label)
    break

tensor([1.1000e+01, 7.0000e+00, 4.0000e+00, 1.2000e+01, 2.0190e+03, 7.1480e+03],
       dtype=torch.float64)
tensor([1.1000e+01, 7.0000e+00, 4.0000e+00, 1.2000e+01, 2.0190e+03, 7.1480e+03])


In [37]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden

In [46]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of GRU model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        
        start_time = time.perf_counter()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            h = h.data
            model.zero_grad()
            
            out, h = model(x.float(), h)
            loss = criterion(out, label.float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.perf_counter()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [47]:
lr = 0.001
gru_model = train(train_loader, lr)

Starting Training of GRU model
Epoch 1......Step: 200/22958....... Average Loss for Epoch: 64565651.84
Epoch 1......Step: 400/22958....... Average Loss for Epoch: 64763617.08
Epoch 1......Step: 600/22958....... Average Loss for Epoch: 64466766.166666664
Epoch 1......Step: 800/22958....... Average Loss for Epoch: 64517487.405
Epoch 1......Step: 1000/22958....... Average Loss for Epoch: 64248603.096
Epoch 1......Step: 1200/22958....... Average Loss for Epoch: 64043974.89666667
Epoch 1......Step: 1400/22958....... Average Loss for Epoch: 63817432.09428571
Epoch 1......Step: 1600/22958....... Average Loss for Epoch: 63549302.455
Epoch 1......Step: 1800/22958....... Average Loss for Epoch: 63411514.70666666
Epoch 1......Step: 2000/22958....... Average Loss for Epoch: 63241102.058
Epoch 1......Step: 2200/22958....... Average Loss for Epoch: 63078735.67818182


KeyboardInterrupt: 